# Implementation of NCSN (Noise Conditional Score Networks)

In [1]:
%load_ext autoreload
%autoreload 2
!nvidia-smi
!which python

Sun Sep 29 15:46:38 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.86.10              Driver Version: 535.86.10    CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla V100-SXM2-32GB           On  | 00000004:04:00.0 Off |                    0 |
| N/A   38C    P0              39W / 184W |      0MiB / 32768MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
try:
    from NCSN.train import train, make_dataset, cal_noise_level, langevin
    from NCSN.utils import train_set, val_set
    from NCSN.model import UNet
except:
    from train import train, make_dataset, cal_noise_level, langevin
    from utils import train_set, val_set
    from model import UNet

from torch.utils.data import DataLoader

import torch
import torch.nn as nn
import torch.nn.functional as F
import datetime
import os

train_loader = DataLoader(train_set, 256, shuffle=True, drop_last=False, pin_memory=True)
val_loader = DataLoader(val_set, 500, shuffle=True, drop_last=False, pin_memory=True)

def timestr():
    now = datetime.datetime.now()
    return now.strftime("%Y%m%d_%H%M%S")

def get_outdir(time_str):
    outdir = f"NCSN/training_data/{time_str}.out"
    return outdir

def get_sample_dir(time_str):
    dir = f"NCSN/samples/{time_str}/"
    if not os.path.exists(dir):
        os.makedirs(dir, exist_ok=True)
    return dir

Files already downloaded and verified
Files already downloaded and verified


In [3]:
# Here are the hyperparameters

epochs = 200
criterion = nn.MSELoss()
init_sigma = 1
final_sigma = 0.01
n_sigma = 10
eps = 2e-5
T = 100

model = UNet(L=n_sigma)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

sigmas = cal_noise_level(init_sigma, final_sigma, n_sigma)

time_str = timestr()

train(epochs=epochs, model=model, optimizer=optimizer, criterion=criterion, train_loader=train_loader, val_loader=val_loader, sigmas=sigmas, eps=eps, T=T, outdir=get_outdir(time_str), eval_freq=1, sample_dir=get_sample_dir(time_str))

Epoch 0: train loss 0.8027304471807277


Epoch 1: train loss 0.5942564124756671


Epoch 2: train loss 0.5071269518517433


Epoch 3: train loss 0.44996082858836395


Epoch 4: train loss 0.40960474965420174


...

Epoch 90: train loss 0.10690671706453282


Epoch 91: train loss 0.10674774145826381


Epoch 92: train loss 0.10636287528149625


Epoch 94/200:  51%|████████████████████████▊                        | 119/235 [00:13<00:12,  8.96it/s, Train Loss=0.106]

: 

In [ ]:
# load a model
import os
model = UNet(L=n_sigma).cuda()
print(sum(p.numel() for p in model.parameters() if p.requires_grad))
eval_epoch = 70
model.load_state_dict(torch.load(f'NCSN/models/{eval_epoch}.pth'))
make_dataset(model, sigmas, eps=eps, T=T)
print("Dataset created")
os.system("python NCSN/evaluate.py")

In [4]:
epochs = 200
criterion = nn.MSELoss()
init_sigma = 1
final_sigma = 0.01
n_sigma = 10
eps = 2e-5
T = 100

sigmas = cal_noise_level(init_sigma, final_sigma, n_sigma)

model = UNet(L=n_sigma).cuda()
eval_epoch = 70
model.load_state_dict(torch.load(f"NCSN/models/{eval_epoch}.pth"))
model.to('cuda')

eps = 2e-5
clamp = False

# x = torch.rand(10, 1, 28, 28).cuda()
x = torch.randn(10, 1, 28, 28).cuda()
x = (x+1)/2
y = langevin(model, x, sigmas, eps=eps, T=T, save=True, epochs=eval_epoch, clamp=clamp)